In [76]:
import astra
import dicom2nifti
import dicom2nifti.compressed_dicom as compressed_dicom
import copy
import importlib
import itertools
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.registration as regs
import niftiutils.private as prv
import niftiutils.visualization as vis
from scipy.ndimage.interpolation import rotate
import numpy as np
import os
import shutil
import pandas as pd
import pylab
import random
import shutil
import config
import time
%matplotlib inline

import glob
from os.path import *

In [77]:
importlib.reload(config)
C = config.Config()

In [13]:
for fn in glob.glob(r"Z:\TACE\Imaging\*\*CT*"):
    shutil.copytree(fn, join(train_dir, basename(fn)))

In [18]:
for fn in glob.glob(join(train_dir,"*")):
    F = basename(fn)
    os.rename(fn, join(dirname(fn), prv.decode(F.strip())))

In [8]:
H = hf.load_dcm_header(r"D:\CBCT\Train\DICOMs\11774179\liver + abd with_2")

In [24]:
importlib.reload(hf)
hf.split_dcm(fn)

In [ ]:
for fn in glob.glob(join(train_dir,"DICOMs","*","*_2")):
    accnum = basename(dirname(fn))
    if exists(join(train_dir,"NIFTIs",accnum+"_2.nii.gz")):
        continue
    try:
        I,D = hf.dcm_load(fn, True, True)
    except:
        print(fn)
        continue
    hf.save_nii(I, join(train_dir,"NIFTIs",accnum+"_2.nii.gz"), D)

In [49]:
for fn in glob.glob(join(train_dir,"DICOMs","*","*_2")):
    try:
        hf.split_dcm(fn);
        if len(os.listdir(fn)) == 0:
            shutil.rmtree(fn)
        time.sleep(.1)
        os.rename(fn+"_bin0", fn)
        I,D = hf.dcm_load(fn, True, True)
    except:
        print(fn)
        continue
    accnum = basename(dirname(fn))
    hf.save_nii(I, join(train_dir,"NIFTIs",accnum+"_2.nii.gz"), D)

In [48]:
for fn in glob.glob(r"D:\CBCT\Train\DICOMs\E102675778\chest_abd  5.0  b31f st_3_*\*"):
    shutil.move(fn, join(r"D:\CBCT\Train\DICOMs\E102675778\chest_abd  5.0  b31f st_3", basename(fn)))

In [3]:
for fn in glob.glob(r"D:\CBCT\Train\MRs\*"):
    os.rename(fn, fn[:fn.rfind('_3')]+"_2.nii.gz")

In [33]:
with open("list.txt", 'r') as f:
    F = [x.strip() for x in f.readlines()]

In [30]:
for fn in F[1:]:
    if len(os.listdir(fn)) == 0:
        shutil.rmtree(fn)
        os.rename(fn+"_bin0", fn)

In [ ]:
for fn in F:
    try:
        hf.dcm_fill_missing_slices(fn);
        I,D = hf.dcm_load(fn, True, True)
    except:
        print(fn)
        continue
    accnum = basename(dirname(fn))
    #P = cm.project(I,D)
    #I = tr.rescale_img(I,D,)
    hf.save_nii(I, join(train_dir,"NIFTIs",accnum+"_2.nii.gz"), D)

In [40]:
fn = glob.glob(join(train_dir,"DICOMs","*","*_3"))[0]

nii_path = random.choice(glob.glob(r"D:\CBCT\Train\NIFTIs\*"))
I,D = hf.nii_load(nii_path)
I = tr.apply_window(I, limits=(-140,500))
I -= I.min()
I = I/I.max()
I = tr.rescale_img(I, [random.random()*1.5+1.5]*3, D)
I = hf.crop_nonzero(I)[0]
I = np.transpose(I, (2,1,0)) # Coordinate order: slice, row, column (z, y, x)
cropI = tr.rescale_img(I, C.world_dims)

proj_id, proj_data = astra.create_sino3d_gpu(cropI, proj_geom, vol_geom) # Coordinate order: row (v), angle, column (u)
proj_data = np.transpose(proj_data, (0,2,1))


In [164]:
pixel_spacing = (1,1)
dim = C.world_dims[0]
rows=cols=sl=dim

aug_factor = 20
d_src = 800
d_det = 0

# Parameter order: rows, colums, slices (y, x, z)
vol_geom = astra.create_vol_geom(rows, cols, sl)

angles = np.linspace(-0.5*np.pi, 0.5*np.pi, 32, False)
# Parameters: width of detector column, height of detector row, #rows, #columns,
#             angles, distance source-origin, distance origin-detector
proj_geom = astra.create_proj_geom('cone', *pixel_spacing, rows, cols, angles, d_src, d_det)

In [192]:
os.remove(nii_path)

In [191]:
[basename(nii_path) for nii_path in glob.glob(r"D:\CBCT\Train\NIFTIs\*")].index("E101349588_3.nii.gz")

219

In [ ]:
for nii_path in glob.glob(r"D:\CBCT\Train\NIFTIs\*")[219:]:
    imgnum = basename(nii_path)
    imgnum = imgnum[:imgnum.find('.')]
    print(imgnum)

    I,D = hf.nii_load(nii_path)
    I = tr.apply_window(I, limits=(-140,500))
    I -= I.min()
    I = I/I.max()
    I = tr.rescale_img(I, [random.random()*1.5+1.5]*3, D)
    I = hf.crop_nonzero(I)[0]
    I = np.transpose(I, (2,1,0)) # Coordinate order: slice, row, column (z, y, x)
    
    aug_save()

E101379629_2


In [185]:
def aug_save():
    for augnum in range(aug_factor):
        cropI = copy.deepcopy(I)
        if random.random() > .5:
            cropI = rotate(cropI, random.random()*360)
        if random.random() > .5:
            cropI = cropI[::-1]
        if random.random() > .5:
            cropI = cropI[:,::-1]
        if random.random() > .5:
            cropI = cropI[...,::-1]
        cropI = rotate(cropI, random.random()*360, axes=(2,1))
        cropI = hf.crop_nonzero(cropI)[0]

        x,y,z = [int(random.random()*(cropI.shape[i]-dim)) for i in range(3)]
        crop = [slice(x,x+dim), slice(y,y+dim), slice(z,z+dim)]
        ix = 0
        while np.percentile(cropI[crop], 50) < .1:
            x,y,z = [int(random.random()*(cropI.shape[i]-dim)) for i in range(3)]
            crop = [slice(x,x+dim), slice(y,y+dim), slice(z,z+dim)]
            ix += 1
            if ix > 10:
                print(nii_path, "not dense enough")
                return
        cropI = cropI[crop]
            
        cropI += np.random.normal(scale=.01, size=cropI.shape)
        cropI -= cropI.min()
        cropI = cropI/cropI.max()
        #cropI = tr.apply_window(cropI, limits=(0,1))

        proj_id, proj_data = astra.create_sino3d_gpu(cropI, proj_geom, vol_geom) # Coordinate order: row (v), angle, column (u)
        proj_data = np.transpose(proj_data, (0,2,1))
        proj_data -= proj_data.min()
        proj_data = proj_data/proj_data.max()

        np.save(join(C.npy_dir, "%s_%d_proj.npy" % (imgnum, augnum)), proj_data)
        np.save(join(C.npy_dir, "%s_%d_img.npy" % (imgnum, augnum)), cropI)

        astra.data3d.delete(proj_id)

In [180]:
for fn in glob.glob(join(C.npy_dir, "*_img.npy")):
    cropI = np.load(fn)
    if np.isnan(cropI.min()):
        print(fn)
    #np.save(fn, cropI)

In [ ]:
    #import pylab
    #pylab.gray()
    #pylab.figure(1)
    #pylab.imshow(proj_data[1,:,:])

    #vis.draw_slices(np.transpose(cropI, (2,1,0)))